<h1 style="margin-bottom:0"><center>DI 501 - Introduction to Data Informatics</center></h1>
<h2 style="margin-top:0"><center>Assignment #3</center></h2>
<h3 style="margin-top:0"><center>Due: December 12, Sunday 23:59</center></h3>
<br>
<p style="margin-top:0"><center><b>This assignment is prepared for DI 501 - Introduction to Data Informatics course offered by  Middle East Technical University.</b></center></p>
<hr style="height:2px;color:navy;margin-top:0">
<p style="margin-top:0; text-align: justify; font-size:15px">The objectives of this assignment are:
    <li>You will be able to apply data preprocessing concepts on a given problem </li>
    <li>You will be able to understand and apply missingness mechanisms and imputation methods to a given problem.</li>
    <li>You will be able to perform data profiling.</li>
    
### Submission and Grading Policy ###
<p style="margin-top:0"><center><b></b></center></p>
  
- Submit your assignments to the corresponding assignment link in https://odtuclass.metu.edu.tr.
- Solutions should be submitted in a single **ipynb** file named: metu-username_HW_2.ipynb (e.g., “e123456_HW_2.ipynb”).
- Late submissions will be accepted by December 15, 2021, 23:59 with a 5% per day penalty.
- Do not change variable names in the code blocks and the texts in the markdowns.
- **<font color=#C91414>This is an individual assignment. Please do not collaborate. You have to adhere to the academic integrity principles.</font>** 
- If needed, you are required to explain your reasonings by adding new blocks below your codes.

<h3 style="margin-bottom:0">Import Libraries</h3>

Here, please import the necessary libraries to conduct your analysis here.

In [54]:
### answer ###
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

<h3 style="margin-bottom:0">1) Data (5 pts)</h3>

We will be utilizing a dataset that is prepared particularly for this exercise. The meaning of individual elements is not important here but we will provide information which will be sufficient for you to conduct your further operations:
- x1: categorical variable that can take values between 0 and 4.
- x2: categorical variable that can take values of low, medium and high.
- x3: categorical variable that can take values of A, B, C and D.
- x4: continuous variable.
- x5: continuous variable.
- y: categorical variable that can take values between 0 and 3.

Please import data with correct delimiter here and retrieve the first 5 rows.

In [55]:
### answer ###
df = pd.read_csv("./data.csv", delimiter=",")
df.head(5)

,x1,x2,x3,x4,x5,y
0,3,MEDIUM,B,71.87353865111545,77.27994181308213,0
1,2,medium,C,76.32969292828467,123.50976228245605,3
2,2,medium,AB,84.72681676261608,94.11303401429247,0
3,1,low,D,90.89495327283925,131.0801255597466,0
4,1,medium,B,91.37755200701073,78.42142573456847,1


Check the datatypes of each column to see whether we need any adjustments.

In [56]:
### answer ###
df.dtypes

x1    object
x2    object
x3    object
x4    object
x5    object
y     object
dtype: object

<h3 style="margin-bottom:0">2) Preprocessing (45 pts)</h3>

<h4 style="margin-bottom:0">a) Numerical Values</h4>

First, convert x1, x4, x5 and y variables into numerical columns. In case of an invalid parsing, the value will be set as NaN. Once you have finished it, drop all the rows that have any null values. At the end, you are supposed to show that you have no null values. You should also use "downcast" parameter of to_numeric function of pandas and give your reasoning on specific choice.

In [70]:
### answer ###

# Since x1 variable can take values between 0 and 4, unsigned integer is sufficient to represent it. 
df["x1"] = pd.to_numeric(df["x1"], errors="coerce", downcast="unsigned")
# Similary, y variable can take values between 0 and 3, unsigned integer is sufficient to represent it.
df["y"] = pd.to_numeric(df["y"], errors="coerce", downcast="unsigned")
# x4 and x5 are continuous variables, hence they must be float.
df["x4"] = pd.to_numeric(df["x4"], errors="coerce", downcast="float")
df["x5"] = pd.to_numeric(df["x5"], errors="coerce", downcast="float")
print("Dataframe dtypes after conversion:")
print(df.dtypes)

print(f"NaN counts before dropping rows with a NaN value:")
print(df.isna().sum())

df = df.dropna(axis=0, how="any", subset=["x1", "x4", "x5", "y"])
print(f"NaN counts after dropping rows with a NaN value:")
print(df.isna().sum())


Dataframe dtypes after conversion:
x1       uint8
x2    category
x3    category
x4     float32
x5     float32
y        uint8
dtype: object
NaN counts before dropping rows with a NaN value:
x1    0
x2    0
x3    0
x4    0
x5    0
y     0
dtype: int64
NaN counts after dropping rows with a NaN value:
x1    0
x2    0
x3    0
x4    0
x5    0
y     0
dtype: int64


<h4 style="margin-bottom:0">b) Categorical Values</h4>

For x2, you have 3 categories. However, due to capitalization and spacing differences, you can see you have more than 3 categories. First, calculate the frequency of each different entry for x2 column.

In [71]:
### answer ###
df["x2"].value_counts()

medium    147
low        31
high       27
Name: x2, dtype: int64

Now, convert all the entries into either "low", "medium" or "high" and show that you only have 3 different categories for that variable. 

In [72]:
### answer ###
def cleanup_x2(text):
    # hacky but works for this dataset
    first_letter = text.strip().lower()[0]
    mapping = {"l": "low", "m": "medium", "h": "high"}
    return mapping[first_letter]

df['x2'] = df['x2'].apply(cleanup_x2)
df['x2'].value_counts()

medium    147
low        31
high       27
Name: x2, dtype: int64

Finally, convert x2 into category data type.

In [73]:
### answer ###
df['x2'] = df['x2'].astype("category").cat.set_categories(['low', 'medium', 'high'], ordered=True)
df['x2'].dtype

CategoricalDtype(categories=['low', 'medium', 'high'], ordered=True)

We also said, x3 can be A, B, C or D. Please find any value that is not either one of them and display them.

In [74]:
### answer ###
invalid_x3_mask = df['x3'].apply(lambda s: s not in {'A', 'B', 'C', 'D'})
print(df.loc[invalid_x3_mask, 'x3'])

Series([], Name: x3, dtype: category
Categories (4, object): ['A' < 'B' < 'C' < 'D'])


Now, drop all the rows that have an inconsistent value for x3.

In [75]:
### answer ###
df = df.drop(axis=0, index=df.index[invalid_x3_mask])
print(df.loc[df['x3'].apply(lambda s: s not in {'A', 'B', 'C', 'D'})])

Empty DataFrame
Columns: [x1, x2, x3, x4, x5, y]
Index: []


Finally, you can convert x3 into a category.

In [76]:
### answer ###
# assuming x3 values are not ordered
df['x3'] = df['x3'].astype("category").cat.set_categories(['A', 'B', 'C', 'D'], ordered=True)
df['x3'].dtype

CategoricalDtype(categories=['A', 'B', 'C', 'D'], ordered=True)

In [79]:
df['x1'] = df['x1'].astype("category")
df['x1'].dtype

CategoricalDtype(categories=[0, 1, 2, 3, 4], ordered=False)

In [80]:
df['y'] = df['y'].astype("category")
df['y'].dtype

CategoricalDtype(categories=[0, 1, 2, 3], ordered=False)

Show all of our dataset that each of our columns have correct datatypes.

In [78]:
### answer ###
df.dtypes

x1       uint8
x2    category
x3    category
x4     float32
x5     float32
y     category
dtype: object

<h4 style="margin-bottom:0">c) Duplicates</h4>

Find <b>full duplicates</b> and return the <b>first occurence</b> of the duplicate values here.

In [83]:
### answer ###
df[df.duplicated(keep='last')]

,x1,x2,x3,x4,x5,y
59,4,high,B,59.337509,47.873280,3
86,1,medium,D,93.266113,108.719963,0


Now, drop full duplicates and do that  with "inplace" and keep the last occurences.

In [87]:
### answer ###
df.drop_duplicates(keep='last', inplace=True)
df[df.duplicated(keep='last')]

,x1,x2,x3,x4,x5,y


<h3 style="margin-bottom:0">3) Dealing with Nulls (50 pts)</h3>

In this section, first construct three different datasets using the following constraints based on the above dataset:

DS1: One with MCAR missingness (at least with 20% missingness in two variables).

DS2: One with MAR missingness  (at least with 20% missingness in two variables).

DS3: One with MNAR missingness (at least with 20% missingness in two variables).

For each of the DS1, DS2, and DS3 datasets, you should:
- Explain your rationale while creating the missing patterns.
- Prove that your data missingness is in the desired form via missingno visualization methods as well as statistical measures (such as cross tabs).

Then, select one of them (DS1, DS2 or DS3) and create a training dataset which should be randomly created with 70% of the whole dataset. The testing dataset should include the remaining 30%. Ensure to distribute your missing tuples equally in both datasets. More specifically, 50% of the missing tuples should appear in training and 50% in the testing dataset. Finally:
- Apply 2 different types of imputation methods other than deleting column or row. Use your training dataset only while working with imputation techniques. For instance, if you prefer to fill a certain missing data with the mean of the variable, you should calculate the mean on the training dataset only. Then use this calculated value while filling the missing value of that corresponding attribute in the testing dataset.
- Compare imputation methods' performance by looking at the correct values on your training and testing dataset separately. For comparison purposes, you can use metrics such as mean squared error, F-score or etc.

You should do those steps in order and you can add as many cells as you want below. However, please use markdown cells regularly to clarify your steps. I will not be able to fully grade you unless I understand what you did in your codes.

Some useful websites: <br>
<a href="https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values"> link </a> <br>
<a href="https://stats.stackexchange.com/questions/462507/how-to-decide-whether-missing-values-are-mar-mcar-or-mnar"> link </a>